In [1]:
import os
import sys
import pprint

from NYISO_Load_Forecast_Data import *
from pf2rec import *

/Users/sergio.dorado/Documents/GitHub_Repositories/01_Maintained/Papers/Power_Flow_Generation_Tool/pf2rec/pf2rec/em_psse/psse.py:12: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  modes = yaml.load(in_file)


# NYISO Power Flow Data

In [2]:
# Absolute path to the `.mo` file of the model
data_path = os.path.abspath(os.path.join(os.getcwd(), "IEEE14"))

path_mo_file = os.path.abspath(os.path.join(data_path, "IEEE14_Base_Case.mo"))

create_pf_records("IEEE14", 
                  path_mo_file, 
                  data_path,
                 openipsl_version = '1.5.0')

In [4]:
# Downloading raw load forecast and actual load data from NYISO website
download_nyiso_data(2020, "_NYISO_Data", print_info = True)

Load forecast '.zip' file download completed up to 10/14/2020
Forecast load '.zip' file organization completed up to 10/14/2020
Actual load '.zip' file download completed up to 10/14/2020
Actual load '.zip' file organization completed up to 10/14/2020


In [1]:
nyiso_zones = 

load_profiles = dict.fromkeys(nyiso_zones)

for lp in load_profiles:
    data = dict.fromkeys(['load', 'best', 'worst'])
    _time_stamps, _load, _best, _worst = get_weekly_behavior("03/31/2020", lp, "_NYISO_Data", False)
    
    # Saving as numpy arrays (for memory)
    data['load'] = np.asarray(_load, dtype = np.float32)
    data['best'] = np.asarray(_best, dtype = np.float32)
    data['worst'] = np.asarray(_worst, dtype = np.float32)
    
    load_profiles[lp] = data
    data = None

NameError: name 'get_weekly_behavior' is not defined

# Generating Power Flows

In [8]:
from GridCal.Engine import ReactivePowerControlMode
from GridCal.Engine.IO.file_handler import FileOpen

#from GridCal.Engine.Devices import *
from GridCal.Engine.Core.multi_circuit import MultiCircuit

from GridCal.Engine.Simulations.PowerFlow.power_flow_worker import PowerFlowOptions, SolverType
from GridCal.Engine.Simulations.PowerFlow.time_series_driver import TimeSeries
from GridCal.Engine.Simulations.PowerFlow.power_flow_driver import PowerFlowDriver

import random

grid = None

# Grid model in GridCal
file_handler = FileOpen(os.path.abspath(os.path.join("IEEE14", 
                                                     "PSSE_Files", 
                                                     "IEEE14_Base_Case.raw")))

# Creating grid object
grid = file_handler.open()

# Getting names of grids and generators
load_names = [load.name for load in grid.get_loads()]
print(f"loads: {load_names}")
gen_names = [gen.name for gen in grid.get_generators()]
print(f"gen_names: {gen_names}")

loads: ['2_1', '3_1', '4_1', '5_1', '6_1', '9_1', '10_1', '11_1', '12_1', '13_1', '14_1']
gen_names: ['1_1 ', '2_1 ', '3_1 ', '6_1 ', '8_1 ']


## Generating Base Case Power Flow

In [9]:
# Power flow options
options = PowerFlowOptions(SolverType.NR,
                           verbose = True,
                           initialize_with_existing_solution = True,
                           multi_core = False,
                           tolerance = 1e-9,
                          max_iter = 99,
                          control_q = ReactivePowerControlMode.Direct)

pf = PowerFlowDriver(grid, options)

pf.run()
    
gridcal2rec(grid = grid, pf = pf, model_name = 'IEEE14', 
    data_path = data_path,
    pf_num = 0, export_pf_results = False, 
    is_time_series = False, openipsl_version = '1.5.0')

Q control logic (fast)
Q controls Ok
Stabilized in 1 iteration(s) (outer control loop)


## Run Time-Series Power Flow with NYISO Data

In [10]:
# Formatting profiles (this maintains the info of the base case)
grid.format_profiles(range(len(_time_stamps)))

seed_value = 0
os.environ['PYTHONHASHSEED'] = str(seed_value)
random.seed(seed_value)

n_matches = min(len(load_names), len(nyiso_zones))

# The `zip` is instantiated as a `list` since it vanishes after one iteration
# Ref: https://stackoverflow.com/questions/28390354/python-zip-object-disappears-after-iterating-through
match_load_area = list(zip(random.sample(load_names, n_matches), random.sample(nyiso_zones, n_matches)))

p_initial = []
q_initial = []
for _load, _ in match_load_area:
    for load in grid.get_loads():
        if load.name == _load:
            p_initial.append(load.P)
            q_initial.append(load.Q)
            
# Power flow options
options = PowerFlowOptions(SolverType.NR,
                           verbose=True,
                           initialize_with_existing_solution = False,
                           multi_core = False,
                           tolerance = 1e-9,
                          max_iter = 99,
                          control_q = ReactivePowerControlMode.Direct)

pf = PowerFlowDriver(grid, options)

# Absolute path to the model
#data_path = os.path.abspath(os.path.join(os.getcwd(), "IEEE14"))

for _scenario in ['best', 'worst', 'load']:
    # Setting power flow profiles
    for n, (_load, _zone) in enumerate(match_load_area):
        for load in grid.get_loads():
            if load.name == _load:
                load.P_prof = p_initial[n] * (load_profiles[_zone][_scenario]/np.max(load_profiles[_zone][_scenario]))
                load.Q_prof = q_initial[n] * (load_profiles[_zone][_scenario]/np.max(load_profiles[_zone][_scenario]))
    
    # Running a power flow
    # The driver needs the power flow options and the grid object as arguments
    for n_ts, ts in enumerate(grid.time_profile):
    
        for load in grid.get_loads():
            load.P = load.P_prof[n_ts]
            load.Q = load.Q_prof[n_ts]
        
        for gen in grid.get_generators():
            gen.P = gen.P_prof[n_ts]
            gen.V = gen.Vset_prof[n_ts]
        
        pf.run()
    
        gridcal2rec(grid = grid, pf = pf, model_name = 'IEEE14', 
            data_path = data_path,
            pf_num = n_ts + 1, export_pf_results = False, 
            is_time_series = True, ts_name = _scenario)

Q control logic (fast)
Q controls Ok
Stabilized in 1 iteration(s) (outer control loop)
Q control logic (fast)
Q controls Ok
Stabilized in 1 iteration(s) (outer control loop)
Q control logic (fast)
Q controls Ok
Stabilized in 1 iteration(s) (outer control loop)
Q control logic (fast)
Q controls Ok
Stabilized in 1 iteration(s) (outer control loop)
Q control logic (fast)
Q controls Ok
Stabilized in 1 iteration(s) (outer control loop)
Q control logic (fast)
Q controls Ok
Stabilized in 1 iteration(s) (outer control loop)
Q control logic (fast)
Q controls Ok
Stabilized in 1 iteration(s) (outer control loop)
Q control logic (fast)
Q controls Ok
Stabilized in 1 iteration(s) (outer control loop)
Q control logic (fast)
Q controls Ok
Stabilized in 1 iteration(s) (outer control loop)
Q control logic (fast)
Q controls Ok
Stabilized in 1 iteration(s) (outer control loop)
Q control logic (fast)
Q controls Ok
Stabilized in 1 iteration(s) (outer control loop)
Q control logic (fast)
Q controls Ok
Stabil

## Exporting List of Power Flows

In [12]:
import os

# Absolute path to the `.mo` file of the model
data_path = os.path.abspath(os.path.join("IEEE14"))

# Path to the `PF_Data` subfolder
pf_data_folder_path = os.path.join(data_path, 'PF_Data')

pf_list = []

# Ref: https://stackabuse.com/python-list-files-in-a-directory/
with os.scandir(pf_data_folder_path) as file_list:
    for file in file_list:
        # print all entries that are files
        if file.is_file():
            if file.name not in ['Power_Flow.mo', 'Power_Flow_Template.mo', 'package.mo', 'package.order']:
                pf_list.append(file.name)
                
df_pf_list = pd.DataFrame(data = pf_list)

df_pf_list.to_csv('pf_list.csv')